In [20]:
import pandas as pd
import os

def get_playlist_data(folder):
    dataframes = {}
    all_files = os.listdir(folder)

    for filename in all_files:
        if filename.endswith('.csv'):
            df = pd.read_csv(os.path.join(folder + '/', filename), sep=';')
            df['wrapped_year'] = filename.split('.')[0]
            df['position'] = df.index + 1
            dataframes[filename] = df
    return dataframes

In [21]:
all_songs = get_playlist_data('wrapped')

all_songs_df = pd.concat(all_songs.values(), ignore_index=True)

all_songs_df.head()

all_songs_df.to_csv('all_songs.csv', sep=';', index=False)

### Genres

In [22]:
genre_info = all_songs_df['Genres']
genres = genre_info.str.split(',').explode().to_frame()

genres.rename(columns={'Genres': 'genre_name'}, inplace=True)
genres.reset_index(drop=True, inplace=True)

genres = genres.dropna()

# Removing duplicates based on 'genre_name'
genres = genres.drop_duplicates(subset='genre_name').reset_index(drop=True)

# Assigning new unique IDs
genres['genre_id'] = genres.index

In [23]:
genres.to_csv('FILES/genres.csv', sep=';', index=False)

### Genre songs

In [24]:
genre_songs = all_songs_df[['Spotify ID', 'Genres']]

# expand the genres column
genre_songs = genre_songs.assign(Genres=genre_songs.Genres.str.split(',')).explode('Genres')

# merge to get genre_id
genre_songs = genre_songs.merge(genres, left_on='Genres', right_on='genre_name', how='left')
genre_songs.drop(columns=['Genres', 'genre_name'], inplace=True)

# datatype of genre_id is float, convert it to int
genre_songs['genre_id'] = genre_songs['genre_id'].astype('Int64')
genre_songs.drop_duplicates(inplace=True)

genre_songs.dropna(inplace=True)

In [25]:
genre_songs.to_csv('FILES/genre_songs.csv', sep=';', index=False)

### Artists

In [26]:
all_songs_df = pd.read_csv('all_songs.csv', sep=';')

In [27]:
artists = all_songs_df[['Artist Name(s)', 'Artist IDs']].copy()

# Create a combined column of tuples (artist name, artist ID)
artists['Artist Info'] = artists.apply(lambda row: list(zip(row['Artist Name(s)'].split(','), row['Artist IDs'].split(','))), axis=1)

# Explode this combined column
exploded_df = artists.explode('Artist Info')

# Create separate columns for artist name and ID from the tuples
exploded_df['Artist Name'] = exploded_df['Artist Info'].apply(lambda x: x[0].strip())
exploded_df['Artist Spotify ID'] = exploded_df['Artist Info'].apply(lambda x: x[1].strip())

# Select only the relevant columns
artist_df = exploded_df[['Artist Name', 'Artist Spotify ID']].drop_duplicates().reset_index(drop=True)

artist_df = artist_df[['Artist Name', 'Artist Spotify ID']].rename(columns={'Artist Name': 'artist_name', 'Artist Spotify ID': 'artist_id'})

In [28]:
artist_df.to_csv('FILES/artists.csv', sep=';', index=False)

### Artist songs

In [29]:
all_songs = all_songs_df.copy()

all_songs['artist_id'] = all_songs_df['Artist IDs'].str.split(',')

# Explode the all_songs_df on 'Artist IDs'
all_songs_exploded = all_songs.explode('artist_id')

merged_df = pd.merge(all_songs_exploded, artist_df, on='artist_id', how='left')

artists_songs = merged_df[['Spotify ID', 'artist_id']].drop_duplicates().reset_index(drop=True)

artists_songs.rename(columns={'Spotify ID': 'song_id', 'artist_id': 'artist_id'}, inplace=True)

artists_songs.to_csv('FILES/artists_songs.csv', sep=';', index=False)

### Playlists

In [30]:
all_songs = get_playlist_data('wrapped')
all_songs_df = pd.concat(all_songs.values(), ignore_index=True)

years = all_songs_df.wrapped_year.unique()
playlists = pd.DataFrame(years, columns=['year'])
playlists['user'] = 'user1'
playlists['playlist_id'] = playlists.index + 1

playlists.to_csv('FILES/playlists.csv', sep=';', index=False)

### Playlist songs

In [31]:
merged = playlists.merge(all_songs_df, how='left', left_on='year', right_on='wrapped_year')
playlist_songs = merged[['playlist_id', 'Spotify ID', 'position']].copy()

playlist_songs.rename(columns={'Spotify ID': 'song_id'}, inplace=True)

playlist_songs.to_csv('FILES/playlist_songs.csv', sep=';', index=False) 

### Songs

In [32]:
songs = all_songs_df.copy()  

songs = songs.drop(columns=['Artist Name(s)', 'Artist IDs', 'Album Name', 'Genres', 'wrapped_year', 'position'])

songs.drop_duplicates(subset='Spotify ID', inplace=True)

songs.rename(columns={'Spotify ID': 'song_id', 'Track Name': 'song_name'}, inplace=True)

songs.to_csv('FILES/songs.csv', sep=';', index=False)   

### Albums

In [33]:
albums = all_songs_df[['Album Name']].copy()

albums['album_id'] = albums.index + 1

albums.drop_duplicates(subset='Album Name', inplace=True)

albums.rename(columns={'Album Name': 'album_name'}, inplace=True)

albums.to_csv('FILES/albums.csv', sep=';', index=False)

### Song albums

In [34]:
songs_albums = all_songs_df[['Spotify ID', 'Album Name']].copy()

songs_albums.rename(columns={'Spotify ID': 'song_id', 'Album Name': 'album_name'}, inplace=True)

songs_albums = songs_albums.merge(albums, how='left', on='album_name')

songs_albums.drop(columns=['album_name'], inplace=True)

songs_albums.drop_duplicates(subset=['song_id', 'album_id'], inplace=True)

songs_albums.to_csv('FILES/songs_albums.csv', sep=';', index=False)

In [35]:


# Fixed the release date column in songs.csv so that it is in the format YYYY-MM-DD

songs = pd.read_csv('FILES/songs.csv', sep=';')

release_date = songs['Release Date'].copy()

# loop over all the release dates
for i in range(len(release_date)):

    date = release_date[i]
    length = len(date)

    if length == 4:
        release_date[i] = date + '-01-01'

    songs['Release Date'] = release_date

songs.to_csv('FILES/songs.csv', sep=';', index=False)

